# Школа Машинного обучения

<a href="https://mipt.ru/science/labs/laboratoriya-neyronnykh-sistem-i-glubokogo-obucheniya/"><img align="right" src="https://avatars1.githubusercontent.com/u/29918795?v=4&s=200" alt="DeepHackLab" style="position:relative;top:-40px;right:10px;height:100px;" /></a>



### Физтех-Школа Прикладной математики и информатики МФТИ  

## Занятие 3. Линейная регрессия
*Ушаков Роман, Захаркин Илья*

Импортируем необходимые библиотеки (и настроим параметры MatPlotLib, чтобы выглядело красиво):

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

titlesize = 20
labelsize = 16
legendsize = labelsize
xticksize = 14
yticksize = xticksize

matplotlib.rcParams['legend.markerscale'] = 1.5 
matplotlib.rcParams['legend.handletextpad'] = 0.5
matplotlib.rcParams['legend.labelspacing'] = 0.4
matplotlib.rcParams['legend.borderpad'] = 0.5
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['font.family'] = 'serif'
matplotlib.rcParams['font.serif'] = 'Times New Roman'
matplotlib.rcParams['axes.labelsize'] = labelsize
matplotlib.rcParams['axes.titlesize'] = titlesize

matplotlib.rc('xtick', labelsize=xticksize)
matplotlib.rc('ytick', labelsize=yticksize)
matplotlib.rc('legend', fontsize=legendsize)

In [38]:
import pandas as pd

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

### Данные

Будем предсказывать стоимость жилья в Бостоне в зависимости от 13 показателей, они описаны ниже (Attributes):

<center><img src="http://arhivach.org/storage/d/31/d31bba5182bc1e93bc1d2e47c6d567cf.jpg" alt="Drawing" width="600" height="100"/></center>

In [6]:
boston = datasets.load_boston()
print(boston.DESCR)

Boston House Prices dataset

Notes
------
Data Set Characteristics:  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive
    
    :Median Value (attribute 14) is usually the target

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pupil-teacher ratio by town
      

Разобьём на тренировочную и тестовую выборки:

**Вопрос:** Зачем это нужно? Почему нельзя обучаться и тестировать на одних и тех же данных?

In [15]:
X = boston.data
y = boston.target
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=42)

Посмотрим на размер тренировочной выборки:

In [17]:
print('Размерность всех данных: {}'.format(boston.data.shape))
print('Размерность тренировочной выборки: {}'.format(X_train.shape))
print('Размерность тустовой выборки: {}'.format(X_test.shape))

Размерность всех данных: (506, 13)
Размерность тренировочной выборки: (404, 13)
Размерность тустовой выборки: (102, 13)


Вот так выглядят данные:

In [12]:
pd.DataFrame(X_train).head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,15.02340,0.0,18.10,0.0,0.6140,5.304,97.3,2.1007,24.0,666.0,20.2,349.48,24.91
1,0.62739,0.0,8.14,0.0,0.5380,5.834,56.5,4.4986,4.0,307.0,21.0,395.62,8.47
2,0.03466,35.0,6.06,0.0,0.4379,6.031,23.3,6.6407,1.0,304.0,16.9,362.25,7.83
3,7.05042,0.0,18.10,0.0,0.6140,6.103,85.1,2.0218,24.0,666.0,20.2,2.52,23.29
4,0.72580,0.0,8.14,0.0,0.5380,5.727,69.5,3.7965,4.0,307.0,21.0,390.95,11.28
5,0.19186,0.0,7.38,0.0,0.4930,6.431,14.7,5.4159,5.0,287.0,19.6,393.68,5.08
6,0.03961,0.0,5.19,0.0,0.5150,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01
7,0.02055,85.0,0.74,0.0,0.4100,6.383,35.7,9.1876,2.0,313.0,17.3,396.90,5.77
8,15.17720,0.0,18.10,0.0,0.7400,6.152,100.0,1.9142,24.0,666.0,20.2,9.32,26.45
9,14.43830,0.0,18.10,0.0,0.5970,6.852,100.0,1.4655,24.0,666.0,20.2,179.36,19.78


### Линейная регрессия без регуляризации

Оптимизируется функционал $$Q = || Y  -  Xw||_2^2 \rightarrow \min\limits_{\theta}$$

Импортируем модуль линейной регрессии

In [18]:
from sklearn.linear_model import LinearRegression

In [19]:
regressor = LinearRegression()

Обучаем модель (оптимизируется функционал выше)

In [20]:
regressor.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

После этого в атрибуте `coef_` будут лежать значения параметра $w$:

In [35]:
print('Количество весов: {}'.format(len(regressor.coef_)))
print('Значения весов: \n{}'.format('\n'.join(['w{} = {}'.format(i+1, x) for i, x in enumerate(regressor.coef_)])))

Количество весов: 13
Значения весов: 
w1 = -0.11246348073596614
w2 = 0.03008101681561436
w3 = 0.040730991914939924
w4 = 2.7867671904193596
w5 = -17.240634709601466
w6 = 4.432487835771647
w7 = -0.006239981732819588
w8 = -1.4484850400834153
w9 = 0.2621137934135512
w10 = -0.010639097815925425
w11 = -0.9163986791910468
w12 = 0.012451646900832003
w13 = -0.5093491195324047


Отклик на новых объектах:

In [36]:
y_pred = regressor.predict(X_test)

Смотрим на метрику на тесте.
Средняя абсолютная ошибка выбрана для того, чтобы показать, на сколько домов мы все-таки ошибаемся:

In [39]:
mean_absolute_error(y_test, y_pred)

3.1915089722747787

### Lasso-регрессия (линейная регрессия с $l_1$-регуляризацией)

Документация: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html#sklearn.linear_model.Lasso

Оптимизируется функционал $$Q = || Y -  Xw||_2^2 +  \lambda \sum\limits_{i = 1}^{n}|w_i| \rightarrow \min\limits_{w}$$

$\lambda$ -- коэффициент регуляризации

Lasso способствует отбору признаков (обнуляет веса при незначимых признаках):  
**Вопрос:** Почему лассо стремится именно обнулить веса, а не просто уменьшить их?

In [40]:
from sklearn.linear_model import Lasso

In [42]:
lasso_regressor = Lasso(alpha=0.4)  # alpha == lambda в формуле выше

Обучаем модель (оптимизируется функционал выше):

In [43]:
lasso_regressor.fit(X_train, y_train)

Lasso(alpha=0.4, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

Отклик на новых объектах:

In [44]:
y_pred = lasso_regressor.predict(X_test)

In [45]:
mean_absolute_error(y_test, y_pred)

3.1405302626371481

**Вопрос:** Почему прирост качества такой незначительный?

### Ridge-регрессия (линейная регрессия с $l_2$-регуляризацией)

Документация: http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge

Оптимизируется функционал $$Q = || Y -  Xw||^2_2 + \lambda||w||_2^2 \rightarrow \min\limits_{w}$$

$\lambda$ -- коэффициент регуляризации

Модифицорванное МНК-pешение

$$w = (X^TX + \lambda I)^{-1}X^TY$$

Гребневая (*ridge* - "гребень") регрессия сокращает веса признаков:

In [46]:
from sklearn.linear_model import Ridge

In [47]:
ridge_regressor = Ridge(alpha=2)

Обучаем модель (оптимизируется функционал выше)

In [48]:
ridge_regressor.fit(X_train, y_train)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

Отклик на новых объектах:

In [49]:
y_pred = ridge_regressor.predict(X_test)

In [50]:
mean_absolute_error(y_test, y_pred)

3.1269167267938651

**Вопрос:** Почему прирост качества такой небольшой?